In [2]:
import pandas as pd
import json

# Configuration
excel_file = "esChat.xlsx"  # Path to your Excel file
output_json = "correct_answers.json"  # Output JSON file
length_list = [3, 6, 9, 12, 15, 18, 21, 24]  # All required snippet lengths

# Read the Excel file
try:
    df = pd.read_excel(excel_file)
    print(f"Successfully loaded Excel file with {len(df)} rows")
    
    # Show column names for debugging
    print(f"\nColumn names:")
    for i, col in enumerate(df.columns):
        print(f"  {i}: '{col}'")
    
except Exception as e:
    print(f"Error loading Excel file: {e}")
    exit(1)

# Filter for mcq == 12 (not 24)
filtered_df = df[df['mcq'] == 12]
print(f"\nFiltered to {len(filtered_df)} rows where mcq == 12")

if len(filtered_df) == 0:
    print("No rows found with mcq == 12. Please check your data.")
    exit(1)

# Check required columns exist
required_columns = ['conversation_id', 'A_label', 'B_label']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"\nERROR: Missing required columns: {missing_columns}")
    print("Available columns:", list(df.columns))
    exit(1)

# Function to convert label to code
def label_to_code(label):
    if pd.isna(label):
        return "?"
    label_str = str(label).strip().lower()
    if 'human' in label_str:
        return 'H'
    elif 'ai' in label_str:
        return 'AI'
    else:
        return label_str.upper()  # Return as-is if not human/ai

# Generate the correct answers dictionary
correct_answers = {}

for _, row in filtered_df.iterrows():
    conv_id = int(row['conversation_id'])
    a_label = label_to_code(row['A_label'])
    b_label = label_to_code(row['B_label'])
    
    # Combine A and B labels
    combined_label = a_label + b_label
    
    print(f"Conversation {conv_id}: A={row['A_label']} -> {a_label}, B={row['B_label']} -> {b_label} = {combined_label}")
    
    # Add entries for all lengths
    for length in length_list:
        key = f"conv{conv_id}_len{length}"
        correct_answers[key] = combined_label

# Sort the dictionary by key for better readability
correct_answers = dict(sorted(correct_answers.items()))

# Save to JSON file
with open(output_json, 'w') as f:
    json.dump(correct_answers, f, indent=2)

print(f"\nGenerated {len(correct_answers)} entries")
print(f"Saved to {output_json}")

# Show a sample of the output
print(f"\nSample entries:")
for i, (key, value) in enumerate(correct_answers.items()):
    if i < 10:  # Show first 10 entries
        print(f'  "{key}": "{value}"')
    elif i == 10:
        print("  ...")
        break

print(f"\nEntries per conversation: {len(length_list)}")
print(f"Total conversations processed: {len(filtered_df)}")

# Summary of label combinations
label_summary = {}
for value in correct_answers.values():
    label_summary[value] = label_summary.get(value, 0) + 1

print(f"\nLabel combinations found:")
for label, count in sorted(label_summary.items()):
    print(f"  {label}: {count} entries")

Successfully loaded Excel file with 210 rows

Column names:
  0: 'conversation_id'
  1: 'Conv_label'
  2: 'mcq'
  3: 'A_label'
  4: 'A_age'
  5: 'A_gender'
  6: 'B_label'
  7: 'B_age'
  8: 'B_gender'
  9: 'A_judgeByB'
  10: 'A_age_judgeByB'
  11: 'A_gender_judgeByB'
  12: 'B_judgeByA'
  13: 'B_age_judgeByA'
  14: 'B_gender_judgebyA'
  15: 'topic'
  16: 'topic_label'

Filtered to 175 rows where mcq == 12
Conversation 35: A=Human -> H, B=Human -> H = HH
Conversation 46: A=Human -> H, B=Human -> H = HH
Conversation 57: A=Human -> H, B=AI -> AI = HAI
Conversation 58: A=Human -> H, B=Human -> H = HH
Conversation 60: A=AI -> AI, B=Human -> H = AIH
Conversation 61: A=Human -> H, B=Human -> H = HH
Conversation 62: A=Human -> H, B=Human -> H = HH
Conversation 63: A=Human -> H, B=Human -> H = HH
Conversation 65: A=AI -> AI, B=Human -> H = AIH
Conversation 67: A=AI -> AI, B=Human -> H = AIH
Conversation 68: A=AI -> AI, B=Human -> H = AIH
Conversation 69: A=AI -> AI, B=Human -> H = AIH
Conversatio